In [13]:
import pandas as pd

df = pd.read_csv("./data/0401/gemma_suppl_test.csv")

df.shape

(359, 3)

In [14]:
# encode
from sentence_transformers import SentenceTransformer

t5_base = SentenceTransformer('sentence-t5-base')

embeddings = t5_base.encode(df['original_text'].tolist()).tolist()

print(len(embeddings))

359


In [15]:
from sklearn.cluster import KMeans
import numpy as np

def diverse_subset(vectors, k):
    # Convert vectors to numpy array
    X = np.array(vectors)
    
    # Perform K-means clustering
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    
    # Get the cluster labels
    cluster_labels = kmeans.labels_
    
    # Find the centroids of each cluster
    centroids = kmeans.cluster_centers_
    
    # Initialize a list to store the diverse subset
    diverse_subset = []
    
    # Select one vector from each cluster to include in the diverse subset
    for i in range(k):
        cluster_indices = np.where(cluster_labels == i)[0]
        if len(cluster_indices) > 0:
            # Choose a representative vector from the cluster (e.g., centroid)
            representative_index = np.argmin(np.linalg.norm(X[cluster_indices] - centroids[i], axis=1))
            diverse_subset.append(cluster_indices[representative_index])
    
    return diverse_subset

subset = diverse_subset(embeddings, 150)
print("Diverse subset:", len(subset))

/opt/miniconda3/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Diverse subset: 150


In [16]:
df.iloc[subset].sample(frac=1).reset_index(drop=True).to_csv("./data/0401/150_suppl_original_text.csv", index=False, columns=['original_text'])